In [1]:
import cv2

In [2]:
# Name of image file
input_image = 'mydoc.jpg'

# Read image
image = cv2.imread(input_image)

# Show image 
cv2.imshow('Input', image)
cv2.waitKey()

-1

In [3]:
# Change image to gray
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Blur the image to erase noise
blur = cv2.blur(gray, (3, 3))

# Find edges with Canny
edge = cv2.Canny(blur, 50, 300, 3)

# Show on screen
cv2.imshow('gray', gray)
cv2.imshow('blur', blur)
cv2.imshow('edge', edge)
cv2.waitKey()

-1

In [4]:
# Find contours
cnts = cv2.findContours(edge, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

import imutils

cnts = imutils.grab_contours(cnts)

# Arrange in decreasing-area order
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)

# Take the 1st contour - the biggest
cnts = cnts[:1]

# Draw contour on orginal image
p = cv2.arcLength(cnts[0], True)
r = cv2.approxPolyDP(cnts[0], 0.02*p, True)
cv2.drawContours(image, [r], -1, (0, 0, 255), 3)

# Show image
cv2.imshow('Draw', image)
cv2.waitKey()

-1

In [5]:
# Reshape ROI into (4,2) 
r = r.reshape(4,2)
 
# Calculate 4 corners: top left, top right, bottem right, bottom left
import numpy as np
rect = np.zeros((4,2), dtype='float32')
 
# Calculate sum for each columns
# top left corner has the smallest sum
# bottom right corner has the biggest sum
s = np.sum(r, axis=1)
rect[0] = r[np.argmin(s)] # top left
rect[2] = r[np.argmax(s)] # bottom right
 
# Calculate difference for each columns
# top right corner has the smallest diff
# bottom left corner has the biggest diff
diff = np.diff(r, axis=1)
rect[1] = r[np.argmin(diff)]
rect[3] = r[np.argmax(diff)]
 
# Calculate height and width of the doc
(tl, tr, br, bl) = rect
 
width1 = np.sqrt((tl[0] - tr[0])**2 + (tl[1] - tr[1])**2 )
width2 = np.sqrt((bl[0] - br[0])**2 + (bl[1] - br[1])**2 )
Width = max(int(width1), int(width2))
 
height1 = np.sqrt((tl[0] - bl[0])**2 + (tl[1] - bl[1])**2 )
height2 = np.sqrt((tr[0] - br[0])**2 + (tr[1] - br[1])**2 )
Height = max(int(height1), int(height2))
 
# New coordination of the doc
new_rect = np.array([[0,0],[Width-1, 0],[Width-1, Height-1],[0, Height-1]], dtype="float32")
 
# Calculate transform matrix
M = cv2.getPerspectiveTransform(rect, new_rect)
 
# Rotate and crop
output = cv2.warpPerspective(image, M, (Width, Height))
 
# Show image
cv2.imshow('Output',output)
cv2.waitKey()

-1

In [6]:
# Turn into gray
gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
 
# Use threshold
 
_, output_final = cv2.threshold(gray,200,455,cv2.THRESH_BINARY)
 
# Show image
cv2.imshow('Output', output_final)
cv2.imwrite('result.jpg', output_final)
cv2.waitKey()

-1